In [3]:
import numpy as np
from numpy import genfromtxt
from math import sqrt
import random
import time

In [4]:
dataset = genfromtxt('kmeans_data/data.csv', delimiter=',')
labels = genfromtxt('kmeans_data/label.csv', delimiter=',')

In [5]:
class Point():
    def __init__(self, features, label):
        self.features = features
        self.label = label

In [6]:
points = np.array([Point(dataset[i], labels[i]) for i in range(len(dataset))])

In [7]:
def euclidean_dist(a, b):
    return sqrt(np.sum((a - b) * (a - b)))
def cosine_dist(a, b):
    num = np.dot(a, b)
    denom = sqrt(np.dot(a, a)) * sqrt(np.dot(b, b))
    return  1 - (num / denom)
def jaccard_dist(a, b):
    less = np.sum(a[a <= b]) + np.sum(b[b < a])
    more = np.sum(a[a >= b]) + np.sum(b[b > a])
    return 1 - (less / more)

In [8]:
def sse_dist(a, b):
    return np.sum((a - b) * (a - b))
def compute_sse(clusters, centroids):
    # Safeguards against the first iteration
    if centroids.shape[0] == 0:
        return float("inf")
    result = 0
    for i in range(len(centroids)):
        centroid = centroids[i]
        cluster = clusters[i]
        for point in cluster:
            result += sse_dist(point.features, centroid)
    return result

In [9]:
def get_accuracy(clusters):
    num_correct = 0
    total = 0
    for cluster in clusters:
        total += len(cluster)
        freq = {}
        for point in cluster:
            if point.label not in freq:
                freq[point.label] = 1
            else:
                freq[point.label] += 1
        max_freq = 0
        for key in freq.keys():
            max_freq = max(max_freq, freq[key])
        num_correct += max_freq
    return num_correct / total

In [10]:
def converged(condition, centroids, prevCentroids, **kwargs):
    verbose = kwargs["verbose"]
    # skips the first iteration
    if kwargs["iteration"] == 1:
        if verbose:
            print('First Iteration')
        return False
    elif condition == "no_change":
        return np.array_equal(prevCentroids, centroids)
    elif condition == "sse_increased":
        clusters = kwargs["clusters"]
        return compute_sse(clusters, centroids) > compute_sse(clusters, prevCentroids)
    elif condition == "preset":
        return kwargs["iteration"] >= kwargs["preset"]
    elif condition == "all":
        no_change = np.array_equal(prevCentroids, centroids)
        if no_change:
            if verbose:
                print('No change in centroids')
            return True
        clusters = kwargs["clusters"]
        if clusters is not None:
            sse_increased = compute_sse(clusters, centroids) > compute_sse(clusters, prevCentroids)
            if sse_increased:
                if verbose:
                    print('SSE has increased')
                return True
        preset = (kwargs["iteration"] >= kwargs["preset"])
        if preset:
            if verbose:
                print('Preset iteration limit exceeded')
            return True
        return False

In [11]:
def kmeans(points, k, condition, distanceFn=None, preset=None, verbose=False):
    start_time = time.time()

    if distanceFn is None:
        print("Error: distance function not defined")
        return
    
    prevCentroids = np.empty(k)
    centroids = np.array([obj.features for obj in np.random.choice(points, k)])
    iteration = 1
    clusters = None
    
    while not converged(condition, centroids, prevCentroids, clusters=clusters, iteration=iteration, preset=preset, verbose=verbose):
        iteration += 1
        if verbose:
            print(iteration)
        
        # Assign each instance to the current centroids
        # and create clusters
        clusters = [[] for _ in range(k)]
        for point in points:
            # find closest centroid to point
            minDistance = distanceFn(point.features, centroids[0])
            minDistanceIndex = 0
            for i in range(1, len(centroids)):
                d = distanceFn(point.features, centroids[i])
                if d < minDistance:
                    minDistance = d
                    minDistanceIndex = i
            clusters[minDistanceIndex].append(point)
        
        # Current centroids become previous centroids
        prevCentroids = centroids
        
        # Recompute centroids in accordance with
        # the newly computed clusters
        NUM_ATTRS = 784
        centroids = np.zeros((k, NUM_ATTRS))
        for i in range(len(clusters)):
            centroid = np.zeros(NUM_ATTRS)
            for point in clusters[i]:
                centroid += point.features
            if len(clusters[i]) > 0:
                centroid /= len(clusters[i])
            centroids[i] = centroid

    end_time = time.time()
    result = {
        "clusters": clusters,
        "centroids": centroids,
        "sse": compute_sse(clusters, centroids),
        "accuracy": get_accuracy(clusters),
        "num_iterations": iteration,
        "time_elapsed": end_time - start_time
    }
    return result

In [12]:
def repeat_kmeans(num_iterations, points, k, condition, distanceFn=None, preset=None, verbose=False):
    sse_vals = np.empty(num_iterations)
    accuracy_vals = np.empty(num_iterations)
    iteration_vals = np.empty(num_iterations)
    time_vals = np.empty(num_iterations)
    for i in range(num_iterations):
        print(f'kmeans call #{i+1}')
        result = kmeans(points, k, condition, distanceFn, preset, verbose)
        sse_vals[i] = result["sse"]
        accuracy_vals[i] = result["accuracy"]
        iteration_vals[i] = result["num_iterations"]
        time_vals[i] = result["time_elapsed"]
    accuracy_vals *= 100
    print(f'Average SSE: {sse_vals.mean()}')
    print(f'Average Accuracy: {accuracy_vals.mean()}')
    print(f'Average # of iterations: {iteration_vals.mean()}')
    print(f'Average time elapsed: {time_vals.mean()} seconds')

In [13]:
print(f'Euclidean kmeans...')
result_euclidean = kmeans(points, 10, distanceFn=euclidean_dist, condition='no_change')
print(f'euclidean kmeans sse: {result_euclidean["sse"]}')
print(f'euclidean kmeans accuracy: {result_euclidean["accuracy"]}')

Euclidean kmeans...
euclidean kmeans sse: 25465664473.15075
euclidean kmeans accuracy: 0.634


In [14]:
print(f'Cosine kmeans...')
result_cosine = kmeans(points, 10, distanceFn=cosine_dist, condition='no_change')
print(f'Cosine kmeans sse: {result_cosine["sse"]}')
print(f'Cosine kmeans accuracy: {result_cosine["accuracy"]}')

Cosine kmeans...
Cosine kmeans sse: 25425779753.475048
Cosine kmeans accuracy: 0.6293


In [15]:
print(f'Jaccard kmeans...')
result_jaccard = kmeans(points, 10, distanceFn=jaccard_dist, condition='no_change')
print(f'Jaccard kmeans sse: {result_jaccard["sse"]}')
print(f'Jaccard kmeans accuracy: {result_jaccard["accuracy"]}')

Jaccard kmeans...
Jaccard kmeans sse: 25414570552.13181
Jaccard kmeans accuracy: 0.602


In [16]:
num_iterations = 5
print(f'Euclidean kmeans over {num_iterations} iterations')
result_euclidean = repeat_kmeans(num_iterations, points, 10, distanceFn=euclidean_dist, condition="all", preset=50, verbose=False)

Euclidean kmeans over 5 iterations
kmeans call #1
kmeans call #2
kmeans call #3
kmeans call #4
kmeans call #5
Average SSE: 25469720347.17402
Average Accuracy: 57.712
Average # of iterations: 47.2
Average time elapsed: 24.425521564483642 seconds


In [17]:
num_iterations = 5
print(f'Cosine kmeans over {num_iterations} iterations')
result_cosine = repeat_kmeans(num_iterations, points, 10, distanceFn=cosine_dist, condition="all", preset=50, verbose=False)

Cosine kmeans over 5 iterations
kmeans call #1
kmeans call #2
kmeans call #3
kmeans call #4
kmeans call #5
Average SSE: 25503593853.237324
Average Accuracy: 60.65599999999999
Average # of iterations: 49.2
Average time elapsed: 17.390534257888795 seconds


In [18]:
num_iterations = 5
print(f'Jaccard kmeans over {num_iterations} iteartions')
result_jaccard = repeat_kmeans(num_iterations, points, 10, distanceFn=jaccard_dist, condition="all", preset=50, verbose=False)

Jaccard kmeans over 5 iteartions
kmeans call #1
kmeans call #2
kmeans call #3
kmeans call #4
kmeans call #5
Average SSE: 25493640721.710945
Average Accuracy: 58.767999999999994
Average # of iterations: 50.0
Average time elapsed: 93.97007846832275 seconds
